(quick-start:top)=
# Quick Start

If you are familiar with using a REST based API already, this page contains some of the most common operations you are likely to want to use with our API. The rest of the guidebook goes through these operations in greater depth.

If you are not familiar with REST APIs, we can recommend reading [Introduction to Populating a Website with API Data ](https://programminghistorian.org/en/lessons/introduction-to-populating-a-website-with-api-data) from the excellent [Programming Historian](https://programminghistorian.org) tutorial site.

## Retrieve a single object record

The simplest operations. If you already know the system number (XXX museum numbers ref?) assigned to the object you want to retrieve, then you can just retrieve it.

In [1]:
import requests
req = requests.get('https://api.vam.ac.uk/v2/museumobject/O54544')
object_data = req.json()
print("The object you requested has the title '%s'" % object_data["record"]["titles"][0]["title"])

The object you requested has the title 'National Photographic Record and Survey'


The full object data response is explained in {doc}`object-fields-full:top`

## Searching for object(s)

In [2]:
import requests
import pandas as pd

object_df = pd.read_csv("https://api.vam.ac.uk/v2/objects/search?q=Derby&page_size=50&response_format=csv")
object_df.head()


,accessionNumber,systemNumber,objectType,_primaryTitle,_primaryPlace,_primaryMaker__name,_primaryMaker__association,_primaryDate,_primaryImageId,_sampleMaterial,_sampleTechnique,_sampleStyle,_currentLocation__displayName,_objectContentWarning,_imageContentWarning
0,E.17-1998,O205723,Photograph,Photograph of Lord Derby,NaN,NaN,NaN,ca. 1860,NaN,NaN,NaN,NaN,"Prints & Drawings Study Room, room 512M",False,False
1,E.386-1921,O705666,Poster,"""Peace"". On The Road To Derby.",England,"Hardy, Dudley",artist,ca.1902,NaN,paper,colour lithography,NaN,"Prints & Drawings Study Room, room 514",False,False
2,C.311-2009,O1173325,Vase,Eastman Imari Vase No.2,Derby,"Eastman, Ken",designer,2009,2009CR0611,bone china,NaN,NaN,"Ceramics, Room 140, Factory Ceramics",False,False
3,C.316-2009,O1173330,Vase,EastmanSnow Leopard Vase No.1,Derby,"Eastman, Ken",designer,2009,2010EH9819,bone china,NaN,NaN,In store,False,False
4,C.313-2009,O1173327,Vase,Eastman Gold Aves Vase No.4,Derby,"Eastman, Ken",designer,2009,2009CR0617,bone china,NaN,NaN,"Ceramics, Room 140, Factory Ceramics",False,False


## Filter object(s)

This lets you filter for objects using identifiers. Much more on this {doc}`filter-objects`. Some common identifiers are listed {doc}`common-identifiers` to get you started.

In [ ]:
import requests
import pandas as pd

object_df = pd.read_csv("https://api.vam.ac.uk/v2/objects/search?id_material=AAT45514&page_size=50&response_format=csv")
object_df.head()

## Cluster objects

If you are more interested in the numbers of objects that match a query than the individual objects themselves, you can use the cluster endpoint which returns counts of objects matching on one or more of the controlled vocabulary fields.

### All cluster types

In [21]:

import requests
import pandas as pd
import altair as alt

req = requests.get('https://api.vam.ac.uk/v2/objects/clusters/search?q=Paris')
object_data = req.json()
object_info = object_data["info"]
record_count = object_info["record_count"]
object_clusters = object_data["clusters"]

print(f"The matching {record_count} object records have the these clusters:")

for cluster_type in object_clusters:
    print("  Cluster type '%s' has the most common value ''%s'" % (cluster_type, object_clusters[cluster_type]["terms"][0]["value"]))


From 73819 matching object records:
  Cluster type 'category' has the most common value ''Designs'
  Cluster type 'person' has the most common value ''Jean-Charles Worth'
  Cluster type 'organisation' has the most common value ''Worth'
  Cluster type 'collection' has the most common value ''Prints, Drawings & Paintings Collection'
  Cluster type 'gallery_location' has the most common value ''Prints & Drawings Study Room, level E'
  Cluster type 'style' has the most common value ''French School'
  Cluster type 'place' has the most common value ''Paris'
  Cluster type 'object_type' has the most common value ''Fashion design'
  Cluster type 'technique' has the most common value ''watercolour drawing'
  Cluster type 'material' has the most common value ''watercolour'
  Cluster type 'maker' has the most common value ''Worth'
  Cluster type 'associated' has the most common value ''Louvre (Paris)'
  Cluster type 'depicts' has the most common value ''Paris'


### One or more cluster type(s)

If you already know the type(s) of cluster terms you want to get (for example materials), you have two endpoint choices. The first passes a 'cluster_type' parameter to the same endpoint as above, and the response is the same but with only the cluster(s)you specified (you can have more than one by repeating the cluster_type param):

In [14]:
import requests
import pandas as pd
import altair as alt

req = requests.get('https://api.vam.ac.uk/v2/objects/clusters/search?cluster_type=material&id_place=x29337&made_after_year=1700&made_before_year=1800&page_size=10')
object_data = req.json()
object_clusters = object_data["clusters"]
materials = object_clusters["material"]["terms"]

materials_df = pd.DataFrame(materials[0:10])
alt.Chart(materials_df, title="Materials used in C18th Welsh objects").mark_bar().encode(
    x=alt.X('value', sort='-y', title='Material'),
    y=alt.Y('count', title='Number of Objects'))

alt.Chart(...)

### Specified cluster type

The second option just returns the term information, which makes it even easy to load into some data science tools such as Pandas:

In [24]:
import pandas as pd
import altair as alt

#materials_df = pd.read_json('http://vam-etc-test-api.azureedge.net/api/v2/objects/clusters/material/search?id_place=x29337&made_after_year=1700&made_before_year=1800&page_size=10')
#alt.Chart(materials_df, title="Materials used in C18th Welsh objects").mark_bar().encode(
#    x=alt.X('value', sort='-y', title='Material'),
#    y=alt.Y('count', title='Number of Objects'))